In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# read all the data
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')
links = pd.read_csv('links.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


**1. POPULARITY BASED RECOMMENDATION**

In [2]:
g = ratings.groupby('userId')['rating'].count()
np.unique(g)


array([  20,   21,   22,   23,   24,   25,   26,   27,   28,   29,   30,
         31,   32,   33,   34,   35,   36,   37,   38,   39,   40,   41,
         42,   43,   44,   45,   46,   47,   48,   50,   51,   52,   53,
         54,   55,   56,   57,   58,   59,   60,   61,   62,   63,   64,
         65,   66,   67,   68,   69,   70,   71,   72,   73,   74,   75,
         76,   77,   78,   80,   81,   82,   83,   84,   85,   86,   87,
         88,   89,   90,   92,   93,   94,   95,   97,   98,  100,  101,
        102,  103,  105,  106,  107,  108,  109,  110,  111,  112,  113,
        114,  115,  118,  119,  120,  121,  122,  123,  126,  127,  128,
        129,  130,  131,  133,  134,  135,  138,  139,  140,  141,  148,
        150,  152,  154,  155,  156,  157,  161,  163,  164,  165,  167,
        168,  173,  174,  176,  177,  179,  180,  181,  185,  186,  187,
        188,  189,  190,  191,  194,  196,  200,  201,  202,  204,  207,
        208,  210,  213,  215,  216,  217,  221,  2

In [3]:
g = ratings.groupby('movieId')['rating'].count()
np.unique(g)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 115, 116, 119, 120, 121,
       122, 123, 124, 125, 126, 128, 129, 131, 132, 133, 135, 136, 137,
       140, 141, 142, 143, 144, 145, 146, 149, 157, 159, 161, 162, 164,
       165, 170, 171, 172, 177, 178, 179, 181, 183, 185, 188, 189, 190,
       192, 196, 198, 200, 201, 202, 203, 204, 211, 215, 218, 220, 224,
       237, 238, 251, 278, 279, 307, 317, 329], dtype=int64)

In [4]:
movie_rating_mean = ratings.groupby('movieId',as_index=False)['rating'].mean()
movie_rating_mean.columns = ['movieId','avg_rating']
movie_rating_count = ratings.groupby('movieId',as_index=False)['rating'].count()
movie_rating_count.columns = ['movieId','votes']
movie_rating_mean_count = movie_rating_mean.merge(movie_rating_count,how='inner',on='movieId')
movie_rating_mean_count


,movieId,avg_rating,votes
0,1,3.920930,215
1,2,3.431818,110
2,3,3.259615,52
3,4,2.357143,7
4,5,3.071429,49
...,...,...,...
9719,193581,4.000000,1
9720,193583,3.500000,1
9721,193585,3.500000,1
9722,193587,3.500000,1


C = movie_rating_mean_count['avg_rating'].mean()
m = movie_rating_mean_count['votes'].quantile(0.9)
def weighted_rating(x, m=1, C=1):
    v = x['votes']
    R = x['avg_rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [6]:
movie_score = movie_rating_mean_count[movie_rating_mean_count['votes']>=m] 
movie_score['score'] = movie_score.apply(weighted_rating,axis=1)
movie_score = movie_score.sort_values('score',ascending=False)
movie_score

C:\Users\santh\AppData\Local\Temp/ipykernel_5560/1322492091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_score['score'] = movie_score.apply(weighted_rating,axis=1)


,movieId,avg_rating,votes,score
277,318,4.429022,317,4.418239
659,858,4.289062,192,4.272021
2224,2959,4.272936,218,4.257991
602,750,4.268041,97,4.234694
921,1221,4.259690,129,4.234615
...,...,...,...,...
1173,1562,2.214286,42,2.186047
2028,2701,2.207547,53,2.185185
1234,1644,2.109375,32,2.075758
1372,1882,1.954545,33,1.926471


**2. CONTENT BASED RECOMMENDER**

In short, we'll recommend content based similar movies. For this dataset, the content can be described with the **genres** and **tags**

We are combining these two in a new dataframe name **movie_metadata**

In [7]:
movie_top_ten = movie_score.head(10)
movie_top_ten = movies[movies['movieId'].isin(movie_top_ten['movieId'])]
# movie_top_ten
movie_top_ten

,movieId,title,genres
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War
659,858,"Godfather, The (1972)",Crime|Drama
686,904,Rear Window (1954),Mystery|Thriller
906,1204,Lawrence of Arabia (1962),Adventure|Drama|War
914,1213,Goodfellas (1990),Crime|Drama
922,1221,"Godfather: Part II, The (1974)",Crime|Drama
2226,2959,Fight Club (1999),Action|Crime|Drama|Thriller
6315,48516,"Departed, The (2006)",Crime|Drama|Thriller


In [8]:
movie_tags = tags.groupby('movieId')['tag'].apply(list).reset_index(name='tag')
movie_tags['tag'] = movie_tags['tag'].apply(lambda x: ' '.join(x))
movie_genres_tags = movies.merge(movie_tags,how='inner',on='movieId')
movie_metadata = movie_genres_tags
movie_metadata['genres'] = movie_metadata['genres'].apply(lambda x: x.replace('|',' '))
movie_metadata['meta'] = movie_metadata['genres']+movie_metadata['tag'] 
movie_metadata= movie_metadata.drop(['genres','tag'],1)
# movie_metadata['meta'] = movie_metadata['meta']
movie_metadata

C:\Users\santh\AppData\Local\Temp/ipykernel_5560/343977015.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movie_metadata= movie_metadata.drop(['genres','tag'],1)


,movieId,title,meta
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasypix...
1,2,Jumanji (1995),Adventure Children Fantasyfantasy magic board ...
2,3,Grumpier Old Men (1995),Comedy Romancemoldy old
3,5,Father of the Bride Part II (1995),Comedypregnancy remake
4,7,Sabrina (1995),Comedy Romanceremake
...,...,...,...
1567,183611,Game Night (2018),Action Comedy Crime HorrorComedy funny Rachel ...
1568,184471,Tomb Raider (2018),Action Adventure Fantasyadventure Alicia Vikan...
1569,187593,Deadpool 2 (2018),Action Comedy Sci-FiJosh Brolin Ryan Reynolds ...
1570,187595,Solo: A Star Wars Story (2018),Action Adventure Children Sci-FiEmilia Clarke ...


In [9]:

from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(stop_words='english')

#Replace NaN with an empty string
movie_metadata['meta'] = movie_metadata['meta'].fillna('')

#Construct the required CountVectorizer matrix by fitting and transforming the data
count_matrix  = count_vectorizer.fit_transform(movie_metadata['meta'])

#Output the shape of count_matrix
count_matrix .shape


(1572, 2375)

Now let's find similarity between each pair of movies. We'll use sklearn's **linear_kernel** for this. So, each of the row, has values of similarities with every other movies.

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim.shape

(1572, 1572)

In [12]:
#Construct a reverse map of indices and movie ids
indices = pd.Series(movie_metadata.index, index=movie_metadata['title'])
indices

title
Toy Story (1995)                         0
Jumanji (1995)                           1
Grumpier Old Men (1995)                  2
Father of the Bride Part II (1995)       3
Sabrina (1995)                           4
                                      ... 
Game Night (2018)                     1567
Tomb Raider (2018)                    1568
Deadpool 2 (2018)                     1569
Solo: A Star Wars Story (2018)        1570
Gintama: The Movie (2010)             1571
Length: 1572, dtype: int64

In [13]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title,movie_metadf,movies_df):
    # title = movie_df[movie_df['movieId']==movie_id]['title'].values[0]
    print('you want recommendation for: ', title)
    # Get the index of the movie that matches the title
    idx = indices[title]
    # print(idx)
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # print(sim_scores)
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    # print(sim_scores)

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    top_movie_ids = movie_metadf['movieId'].iloc[movie_indices].reset_index()
    
    return movie_metadata['title'].iloc[movie_indices]
    # Return the top 10 most similar movies
    # return top_movies

In [14]:
get_recommendations('Batman Returns (1992)',movie_metadata,movies)

you want recommendation for:  Batman Returns (1992)


43                         Batman Forever (1995)
113                   In the Line of Fire (1993)
360    Butch Cassidy and the Sundance Kid (1969)
383                                  Jaws (1975)
384                              Jaws 3-D (1983)
411                             G.I. Jane (1997)
412                         Air Force One (1997)
540                            Siege, The (1998)
553                             Rocky III (1982)
554                              Rocky IV (1985)
Name: title, dtype: object

**3. COLLABORATIVE FILTERING**

In [15]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


In [16]:
from surprise import Reader, Dataset, SVD, model_selection
reader = Reader()
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [17]:
# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
algo = SVD()

In [18]:
model_selection.cross_validate(algo,data,measures=['RMSE','MAE'])

{'test_rmse': array([0.86843187, 0.87233007, 0.88081523, 0.88024695, 0.86746245]),
 'test_mae': array([0.66760035, 0.67098214, 0.67604209, 0.67391332, 0.66780013]),
 'fit_time': (0.9558148384094238,
  1.2109448909759521,
  1.0139358043670654,
  1.0808830261230469,
  1.1231138706207275),
 'test_time': (0.22045254707336426,
  0.10985589027404785,
  0.22253203392028809,
  0.10976195335388184,
  0.25803279876708984)}

In [21]:
# taken from surprise docs
import collections
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = collections.defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [20]:
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)
# predictions
top_n = get_top_n(predictions, n=10)
top_n

defaultdict(list,
            {1: [(318, 5),
              (6874, 5),
              (58559, 5),
              (720, 5),
              (912, 5),
              (1719, 5),
              (5618, 5),
              (7153, 5),
              (922, 5),
              (858, 5)],
             2: [(7361, 4.511561356659342),
              (912, 4.45812271654269),
              (898, 4.446783204076157),
              (1213, 4.444385770778193),
              (2959, 4.437676747013013),
              (49272, 4.4068478637789505),
              (1104, 4.406393626451127),
              (1178, 4.406278335239924),
              (904, 4.39321033188432),
              (106100, 4.374042431587645)],
             3: [(2542, 3.729013102433669),
              (2324, 3.662241295578535),
              (1266, 3.61443363407034),
              (3213, 3.550558954832926),
              (1246, 3.5295094183797286),
              (1288, 3.459236658624256),
              (318, 3.45741837235556),
              (1199, 3.45163668

In [22]:
print_count = 10
cur_p_count = 0
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    cur_p_count+=1
    if cur_p_count>=print_count:
      break

1 [318, 6874, 58559, 720, 912, 1719, 5618, 7153, 922, 858]
2 [7361, 912, 898, 1213, 2959, 49272, 1104, 1178, 904, 106100]
3 [2542, 2324, 1266, 3213, 1246, 1288, 318, 1199, 1196, 117590]
4 [106642, 1208, 1411, 6, 741, 4644, 1278, 1089, 1247, 916]
5 [48516, 1204, 1213, 1197, 7090, 904, 1210, 914, 1035, 5618]
6 [928, 2160, 33166, 1233, 7143, 44555, 7090, 1272, 56782, 1276]
7 [1204, 527, 1250, 1258, 318, 1198, 1272, 296, 933, 910]
8 [1196, 1210, 2959, 1208, 7361, 7153, 3451, 2324, 260, 1204]
9 [2571, 3275, 318, 1242, 1196, 110, 1197, 593, 1204, 6787]
10 [112852, 475, 318, 1704, 50, 910, 1281, 1207, 1035, 1148]
